In [ ]:
!pip install -U scikit-learn

In [ ]:
!pip install  plotly

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import tensorflow.keras.optimizers.schedules as schedules

In [ ]:
epochs=30
rootPath="../input/plant-pathology-2020-fgvc7/"
images= "images/"
test= "test.csv"
train = "train.csv"
result = "sample_submission.csv"

submission = pd.read_csv(rootPath+result)
testData = pd.read_csv(rootPath+test)
trainData = pd.read_csv(rootPath+train)

In [ ]:
def path(st):
    return rootPath + '/images/' + st + '.jpg'

testPaths = testData.image_id.apply(path).values
trainPaths = trainData.image_id.apply(path).values

trainLabels = np.float32(trainData.loc[:, 'healthy':'scab'].values)
trainPaths, validPaths, trainLabels, validLabels =train_test_split(trainPaths, trainLabels, test_size=0.15)

In [ ]:
def dataPreprocessing(filename, label=None, image_size=(512, 512)):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    if label is None:
        return image
    return image, label
    

def dataAugment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if label is None:    
        return image
    return image, label

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

batchSize = 16 * strategy.num_replicas_in_sync

In [ ]:
trainDataset = (
    tf.data.Dataset
    .from_tensor_slices((trainPaths, trainLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .map(dataAugment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(batchSize)
    .prefetch(AUTO)
)

validDataset = (
    tf.data.Dataset
    .from_tensor_slices((validPaths, validLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
    .cache()
    .prefetch(AUTO)
)

testDataset = (
    tf.data.Dataset
    .from_tensor_slices(testPaths)
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
)


In [ ]:
from tensorflow.keras.models import clone_model
class Combination():
    
    def __init__(self):
        self.model=None
        self.optimizer=None
    
    def setModel(self,Model):
        model=Model
        self.model=model.getModel()
        print(self.model.summary())
    
    def setOptimizer(self,Optimizer):
        
        self.optimizer=Optimizer
        print(self.optimizer)
    
    
        
        
    
    def onLearning(self,epochs=30):
        
        with strategy.scope():
        


            
            self.model.compile(optimizer=self.optimizer,
                          loss = 'categorical_crossentropy',
                          metrics=['categorical_accuracy'])
        
        stepPerEpoch = trainLabels.shape[0] // batchSize
        #lrSchedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
        history = self.model.fit(
                    trainDataset,
                    epochs=epochs,
                    #callbacks=[lrSchedule],
                    steps_per_epoch=stepPerEpoch,
                    validation_data=validDataset)
        return history

In [ ]:
class Model:
    def __init__(self):
        pass
    def getModel():
        pass

In [ ]:
class Optimizer:
    def __init__(self):
        pass
    def getOptimizer():
        pass

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications import ResNet152V2

In [ ]:
class Vgg16(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([VGG16(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Vgg19(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([VGG19(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet50(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet50(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet101(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet101(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet152(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet152(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet50v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet50V2(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet101v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet101V2(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet152v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet152V2(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class SGD(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.SGD(momentum=0.9,learning_rate=schedules)
        return self.Optim

In [ ]:
class Adagrad(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.Adagrad(epsilon=1e-6,learning_rate=schedules)
        return self.Optim

In [ ]:
class RMSprop(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.RMSprop(rho=0.9, epsilon=1e-06,learning_rate=schedules)
        return self.Optim
    
        

In [ ]:
class Adam(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999,learning_rate=schedules)
        return self.Optim

In [ ]:
models={"Vgg16":Vgg16(),"Vgg19":Vgg19(),"Resnet50":Resnet50(),\
        "Resnet101":Resnet101(),"Resnet152":Resnet152(),"Resnet50v2"\
        :Resnet50v2(),"Resnet101v2":Resnet101v2(),"Resnet152v2":Resnet152v2()}

In [ ]:
optimizers={"SGD":SGD(),"Adagrad":Adagrad(),"RMSprop":RMSprop(),"Adam":Adam()}

In [ ]:
learningRateSchedulers={"CosineDecay":schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=1000, alpha=0.0)\
                       ,"CosineDecayRestarts":schedules.CosineDecayRestarts(initial_learning_rate=0.001, t_mul=2.0,m_mul=1.0,first_decay_steps=1000, alpha=0.001)\
                       ,"ExponentialDecay":schedules.ExponentialDecay(initial_learning_rate=0.01,decay_steps=50,decay_rate=0.96,staircase=True)\
                       ,"InverseTimeDecay":schedules.InverseTimeDecay(initial_learning_rate = 0.01,decay_steps = 1.0,decay_rate = 0.5)
                       }

In [ ]:
combination=Combination()

In [ ]:
def display(training, validation, title,yTitle):     
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=np.arange(1, 30+1), mode='lines+markers', y=training, marker=dict(color="#dc143c"),name="Train"))

        fig.add_trace(
            go.Scatter(x=np.arange(1, 30+1), mode='lines+markers', y=validation, marker=dict(color="#0080ff"),
                   name="Validation"))

        fig.update_layout(title_text=title, yaxis_title=yTitle, xaxis_title="Epochs", template="plotly_dark")
        fig.show()

In [ ]:

def performanceCheck():
    for modelName,model in models.items():
        weights=model.getModel().get_weights()
        for optimizerName,optimizer in optimizers.items():
            for learningRateSchedulerName,learningRateScheduler in learningRateSchedulers.items():
                model.getModel().set_weights(weights)
                combination.setModel(model)
                combination.setOptimizer(optimizer.setSchedules(learningRateScheduler))
                history=combination.onLearning(epochs=30)
                trainAcc=history.history['categorical_accuracy']
                evalAcc=history.history['val_categorical_accuracy']
                trainLoss=history.history['loss']
                evalLoss=history.history['val_loss']
                title=modelName+" "+optimizerName+" "+learningRateSchedulerName+" "
                display(trainAcc,evalAcc,title+"Accuracy","Accuracy")
                display(trainLoss,evalLoss,title+"Loss","Loss")